In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import spectrogram, chirp
from scipy.signal import firwin, lfilter

# --------------------------------------------------------------------
# 1. Parameters (Same as before)
# --------------------------------------------------------------------
fs = 100_000           # 100 kHz sampling rate
duration = 1.0         # 1 second
t = np.arange(0, duration, 1 / fs)

wave_types = ['bpsk', 'qpsk', 'chirp', 'ofdm']
rng = np.random.default_rng(seed=10)
center_freqs = rng.uniform(3_000, 38_000, size=4)
amplitudes = rng.uniform(0.3, 1.2, size=4)
order = rng.permutation(4)
slot_len = duration / 4
events = []
for idx, slot_id in enumerate(order):
    start = idx * slot_len
    end = start + slot_len * 0.8
    f0 = center_freqs[slot_id]
    amp = amplitudes[slot_id]
    wtype = wave_types[slot_id]
    events.append((start, end, f0, amp, wtype))

# --------------------------------------------------------------------
# 2. Build composite IQ signal with noise
# --------------------------------------------------------------------
x = np.zeros_like(t, dtype=complex)
noise_power = 0.02  # slightly reduced noise for better visibility
x += np.sqrt(noise_power / 2) * (rng.standard_normal(x.size) + 1j * rng.standard_normal(x.size))

for start, end, f0, amp, wtype in events:
    idx_mask = (t >= start) & (t < end)
    tt = t[idx_mask] - start
    dur = end - start
    phase0 = 2 * np.pi * rng.random()

    if wtype == 'bpsk':
        sym_rate = 500
        sym_samp = int(fs / sym_rate)
        num_sym = int(np.ceil(tt.size / sym_samp))
        bits = rng.choice([-1, 1], size=num_sym)
        bpsk_seq = np.repeat(bits, sym_samp)[:tt.size]
        carrier = np.exp(1j * (2 * np.pi * f0 * tt + phase0))
        x[idx_mask] += amp * bpsk_seq * carrier

    elif wtype == 'qpsk':
        sym_rate = 500
        sym_samp = int(fs / sym_rate)
        num_sym = int(np.ceil(tt.size / sym_samp))
        bits_i = rng.choice([-1, 1], size=num_sym)
        bits_q = rng.choice([-1, 1], size=num_sym)
        qpsk_seq = (bits_i + 1j * bits_q) / np.sqrt(2)
        qpsk_seq = np.repeat(qpsk_seq, sym_samp)[:tt.size]
        carrier = np.exp(1j * (2 * np.pi * f0 * tt + phase0))
        x[idx_mask] += amp * qpsk_seq * carrier

    elif wtype == 'chirp':
        k = 4000 / dur
        inst_phase = 2 * np.pi * (f0 * tt + 0.5 * k * (tt ** 2))
        x[idx_mask] += amp * np.exp(1j * (inst_phase + phase0))

    elif wtype == 'ofdm':
        num_subcarriers = 8
        spacing = 100
        ofdm_sig = np.zeros_like(tt, dtype=complex)
        for k_sub in range(num_subcarriers):
            fk = f0 + (k_sub - num_subcarriers // 2) * spacing
            ph = 2 * np.pi * rng.random()
            sign = rng.choice([-1, 1])
            ofdm_sig += sign * np.exp(1j * (2 * np.pi * fk * tt + ph))
        ofdm_sig /= np.sqrt(num_subcarriers)
        x[idx_mask] += amp * ofdm_sig

# --------------------------------------------------------------------
# 3. Correct Spectrogram using only positive frequencies
# --------------------------------------------------------------------
nfft = 512
noverlap = nfft // 2

# Compute spectrogram on complex x (IQ)
freq_bins, time_bins, Sxx = spectrogram(
    x,          # complex input for full IQ power
    fs=fs,
    nperseg=nfft,
    noverlap=noverlap,
    mode='complex',
)

c:\Users\hyunwon\AppData\Local\anaconda3\Lib\site-packages\scipy\signal\_spectral_py.py:1870: UserWarning: Input data is complex, switching to return_onesided=False
  warnings.warn('Input data is complex, switching to '


In [4]:
a = Sxx[0]

In [ ]:
import numpy as np

def complex128_to_float16(arr: np.ndarray) -> np.ndarray:
    """
    Converts a complex128 NumPy array to a float16 array where the last dimension is 2:
    [..., 2] = [real, imag]

    Parameters:
        arr (np.ndarray): Input complex128 array of any shape

    Returns:
        np.ndarray: float16 array with shape [..., 2]
    """
    if not np.issubdtype(arr.dtype, np.complexfloating):
        raise ValueError("Input array must be of complex dtype.")

    real_part = arr.real.astype(np.float16)
    imag_part = arr.imag.astype(np.float16)

    # Expand shape: original_shape + (2,)
    stacked = np.stack((real_part, imag_part), axis=-1)
    return stacked

def complex128_to_binary_array(arr: np.ndarray) -> np.ndarray:
    """
    Converts a complex128 NumPy array into a uint16 array representing the binary
    encoding of float16 values for both real and imaginary parts.

    Output shape: [..., 2], dtype: uint16
    """
    if not np.issubdtype(arr.dtype, np.complexfloating):
        raise ValueError("Input must be a complex array.")

    # 1. 실수/허수 추출 후 float16으로 변환
    real_part = arr.real.astype(np.float16)
    imag_part = arr.imag.astype(np.float16)

    # 2. (N, ..., 2) 형식으로 쌓기
    float16_stacked = np.stack((real_part, imag_part), axis=-1)

    # 3. float16 → uint16 binary representation
    binary_uint16 = float16_stacked.view(np.uint16)

    return binary_uint16

b = complex128_to_float16(a)
c = complex128_to_binary_array(a)

In [33]:
import numpy as np

def complex128_to_bitarray_and_file(arr: np.ndarray, filename: str) -> np.ndarray:
    """
    Converts complex128 array → float16 (real/imag) → binary bit array (0/1),
    and saves the raw uint16 binary data to a .bin file.

    Parameters:
        arr (np.ndarray): Input complex128 array
        filename (str): File path to save binary

    Returns:
        np.ndarray: Binary bit array of shape [..., 2, 16], values in {0, 1}
    """
    if not np.issubdtype(arr.dtype, np.complexfloating):
        raise ValueError("Input must be a complex array.")

    # Step 1: float16 변환 후 쌓기
    real = arr.real.astype(np.float16)
    imag = arr.imag.astype(np.float16)
    stacked = np.stack((real, imag), axis=-1)  # shape [..., 2]

    # Step 2: view as uint16, and save raw binary file
    binary_uint16 = stacked.view(np.uint16)
    binary_uint16.tofile(filename)
    print(f"Saved raw binary to {filename} | shape={binary_uint16.shape}, dtype=uint16")

    # Step 3: convert to bit array (16-bit per value)
    flat_uint16 = binary_uint16.flatten()
    bit_array = np.unpackbits(flat_uint16.view(np.uint8)).reshape(-1, 16)

    # Step 4: reshape back to [..., 2, 16]
    final_shape = (*arr.shape, 2, 16)
    return bit_array.reshape(final_shape)

d = complex128_to_bitarray_and_file(a, "complex_float16.bin")


Saved raw binary to complex_float16.bin | shape=(389, 2), dtype=uint16


In [40]:
import numpy as np

def complex128_to_bitstring_textfile_csvstyle(arr: np.ndarray, filename: str) -> np.ndarray:
    """
    Converts a complex128 array to float16 (real/imag), formats each as 16-bit binary strings,
    and writes them to a text-based .bin file with format: 'real_bits,imag_bits' per line.

    Parameters:
        arr (np.ndarray): complex128 input array
        filename (str): Output file name (e.g., 'output.bin')

    Returns:
        np.ndarray: Array of shape [..., 2] with 16-bit binary strings (dtype=object)
    """
    if not np.issubdtype(arr.dtype, np.complexfloating):
        raise ValueError("Input must be complex dtype.")

    # Step 1: Convert to float16
    real = arr.real.astype(np.float16)
    imag = arr.imag.astype(np.float16)
    stacked = np.stack((real, imag), axis=-1)  # shape [..., 2]

    # Step 2: View as uint16
    uint16_view = stacked.view(np.uint16)  # shape [..., 2]

    # Step 3: Flatten and convert each pair to string
    reshaped = uint16_view.reshape(-1, 2)
    bit_lines = [f"{r:016b},{i:016b}" for r, i in reshaped]

    # Step 4: Save to file as text
    with open(filename, "w") as f:
        for line in bit_lines:
            f.write(line + "\n")

    print(f"✅ Saved {len(bit_lines)} lines to {filename} in 'real_bits,imag_bits' format.")

    # Step 5: Also return as ndarray of bitstrings
    bitstring_array = np.array([[f"{r:016b}", f"{i:016b}"] for r, i in reshaped], dtype=object)
    return bitstring_array.reshape(*arr.shape, 2)

bit_array = complex128_to_bitstring_textfile_csvstyle(a, "complex_values.bin")
bit_array[0][0]

✅ Saved 389 lines to complex_values.bin in 'real_bits,imag_bits' format.


'0000011011000110'